# Physics engines

Gazebo has interfaces with four physics engines

* ODE
* Simbody
* Bullet
* DART

from which only DART has to be compiled separately.
The modules presented below allow the generation of the necessary physics engine's SDF parameters using classes.

## Physics engine object

The physics engine object holds the global parameters valid for all physics engines. 

If this modules is used to generate the SDF data, the default parameters of the physics engine chosen (`ode`, `bullet`, `simbody` or `dart`) will be used.

In [1]:
from pcg_gazebo.simulation.physics import Physics
physics = Physics()

# Iterate through all parameters
for name in physics.get_parameter_names():
    print('{}: {}'.format(name, physics.get_parameter(name)))
    

max_step_size: 0.001
real_time_factor: 1
real_time_update_rate: 1000
max_contacts: 20
engine: ode
name: default_physics
default: False


In [2]:
# The description of the parameters is also available
for name in physics.get_parameter_names():
    physics.print_description(name)

max_step_size
  Description: Maximum time step size at which every system in simulation can interact with the states of the world
  Current value: 0.001
real_time_factor
  Description: Target simulation speedupfactor, defined by ratio of simulation time to real-time
  Current value: 1
real_time_update_rate
  Description: Rate at which to update the physics engine (UpdatePhysics calls per real-time second)
  Current value: 1000
max_contacts
  Description: Maximum number of contactsallowed between two entities. This value can be over ridden by a max_contacts element in a collision element
  Current value: 20
engine
  Description: The type of the dynamics engine. Current options are ode, bullet, simbody and dart. Defaults to ode if left unspecified.
  Current value: ode
name
  Description: The name of this set of physics parameters
  Current value: default_physics
default
  Description: If true, this physics element is set as the default physics profile for the world. If multiple default 

In [3]:
# It is also possible to generate the SDF files.
# The SDF object can also be retrieved and altered if necessary
print(physics.to_sdf('physics'))


<physics name="default_physics" default="1" type="ode">
  <max_step_size>0.001</max_step_size>
  <real_time_factor>1.0</real_time_factor>
  <real_time_update_rate>1000.0</real_time_update_rate>
  <max_contacts>20</max_contacts>
</physics>



In [4]:
print(physics.to_sdf('world'))

<world name="default">
  <physics name="default_physics" default="1" type="ode">
    <max_step_size>0.001</max_step_size>
    <real_time_factor>1.0</real_time_factor>
    <real_time_update_rate>1000.0</real_time_update_rate>
    <max_contacts>20</max_contacts>
  </physics>
  <gravity>0 0 -9.8</gravity>
  <include>
    <uri>model://ground_plane</uri>
  </include>
  <include>
    <uri>model://sun</uri>
  </include>
</world>



In [5]:
# Let's make a custom parameter set for the physics engine
physics.max_step_size = 0.01
physics.real_time_factor = 1
physics.real_time_update_rate = 500
physics.max_contacts = 5
physics.name = 'custom_physics'

print(physics.to_sdf())

<physics name="custom_physics" default="1" type="ode">
  <max_step_size>0.01</max_step_size>
  <real_time_factor>1.0</real_time_factor>
  <real_time_update_rate>500.0</real_time_update_rate>
  <max_contacts>5</max_contacts>
</physics>



In [6]:
# The resulting world file can be exported to an .world file and run in Gazebo
# This shows how to create an SDF file from stratch
sdf = physics.to_sdf('sdf')
print(sdf)

<sdf version="1.6">
  <world name="default">
    <physics name="custom_physics" default="1" type="ode">
      <max_step_size>0.01</max_step_size>
      <real_time_factor>1.0</real_time_factor>
      <real_time_update_rate>500.0</real_time_update_rate>
      <max_contacts>5</max_contacts>
    </physics>
    <gravity>0 0 -9.8</gravity>
    <include>
      <uri>model://ground_plane</uri>
    </include>
    <include>
      <uri>model://sun</uri>
    </include>
  </world>
</sdf>



In [7]:
# Export to a .world file
world_filename = '/tmp/physics.world'
sdf.export_xml(filename=world_filename)

In [8]:
# Start a PCG server to run the world file in Gazebo
from pcg_gazebo.task_manager import Server
server = Server()
# Create a simulation manager named default
server.create_simulation('default')
simulation = server.get_simulation('default')
# Run an instance of the empty.world scenario
# This is equivalent to run
#      roslaunch gazebo_ros empty_world.launch
# with the parameters provided to run the world file created
simulation.create_gazebo_task(
    name='gazebo',
    world=world_filename,
    gui=True,
    physics='ode',
    paused=False,
    required=True,
    process_timeout=10)
# A task named 'gazebo' the added to the tasks list
print(simulation.get_task_list())
# But it is still not running
print('Is Gazebo running: {}'.format(simulation.is_task_running('gazebo')))
# Run Gazebo
simulation.run_all_tasks()

['gazebo']
Is Gazebo running: False


In [9]:
# Check if the parameters were initialized correctly
# We need a Gazebo proxy object to check it
from pcg_gazebo.task_manager import GazeboProxy
# Create a Gazebo proxy
gazebo_proxy = simulation.get_gazebo_proxy()

# It is important to note that the default get_physics_properties service from Gazebo
# returns only the global and the ODE engine parameters
print(gazebo_proxy.get_physics_properties())

time_step: 0.01
pause: False
max_update_rate: 500.0
gravity: 
  x: 0.0
  y: 0.0
  z: -9.8
ode_config: 
  auto_disable_bodies: False
  sor_pgs_precon_iters: 0
  sor_pgs_iters: 50
  sor_pgs_w: 1.3
  sor_pgs_rms_error_tol: 0.0
  contact_surface_layer: 0.001
  contact_max_correcting_vel: 100.0
  cfm: 0.0
  erp: 0.2
  max_contacts: 5
success: True
status_message: "GetPhysicsProperties: got properties"


In [10]:
simulation.wait()
simulation.kill_all_tasks()

## ODE

It is possible to create an instance of the physics engine configuration for each engine available.

The ODE is presented on the following sections.

In [11]:
from pcg_gazebo.simulation.physics import ODE
physics = ODE()

# Iterate through all parameters
for name in physics.get_parameter_names():
    print('{}: {}'.format(name, physics.get_parameter(name)))

max_step_size: 0.001
real_time_factor: 1
real_time_update_rate: 1000
max_contacts: 20
engine: ode
name: default_physics
default: False
min_step_size: 0.0001
iters: 50
sor: 1.3
type: quick
precon_iters: 0
use_dynamic_moi_rescaling: False
friction_model: pyramid_model
cfm: 0
erp: 0.2
contact_surface_layer: 0.001
contact_max_correcting_vel: 100


In [12]:
# The description of the parameters is also available
for name in physics.get_parameter_names():
    physics.print_description(name)

max_step_size
  Description: Maximum time step size at which every system in simulation can interact with the states of the world
  Current value: 0.001
real_time_factor
  Description: Target simulation speedupfactor, defined by ratio of simulation time to real-time
  Current value: 1
real_time_update_rate
  Description: Rate at which to update the physics engine (UpdatePhysics calls per real-time second)
  Current value: 1000
max_contacts
  Description: Maximum number of contactsallowed between two entities. This value can be over ridden by a max_contacts element in a collision element
  Current value: 20
engine
  Description: The type of the dynamics engine. Current options are ode, bullet, simbody and dart. Defaults to ode if left unspecified.
  Current value: ode
name
  Description: The name of this set of physics parameters
  Current value: default_physics
default
  Description: If true, this physics element is set as the default physics profile for the world. If multiple default 

In [13]:
# It is also possible to generate the SDF files.
# The SDF object can also be retrieved and altered if necessary
print(physics.to_sdf('physics'))

<physics name="default_physics" default="1" type="ode">
  <max_step_size>0.001</max_step_size>
  <real_time_factor>1.0</real_time_factor>
  <real_time_update_rate>1000.0</real_time_update_rate>
  <max_contacts>20</max_contacts>
  <ode>
    <solver>
      <min_step_size>0.0001</min_step_size>
      <iters>50</iters>
      <sor>1.3</sor>
      <type>quick</type>
      <precon_iters>0</precon_iters>
      <use_dynamic_moi_rescaling>0</use_dynamic_moi_rescaling>
      <friction_model>pyramid_model</friction_model>
    </solver>
    <constraints>
      <cfm>0.0</cfm>
      <erp>0.2</erp>
      <contact_surface_layer>0.001</contact_surface_layer>
      <contact_max_correcting_vel>100.0</contact_max_correcting_vel>
    </constraints>
  </ode>
</physics>



In [14]:
print(physics.to_sdf('world'))

<world name="default">
  <physics name="default_physics" default="1" type="ode">
    <max_step_size>0.001</max_step_size>
    <real_time_factor>1.0</real_time_factor>
    <real_time_update_rate>1000.0</real_time_update_rate>
    <max_contacts>20</max_contacts>
    <ode>
      <solver>
        <min_step_size>0.0001</min_step_size>
        <iters>50</iters>
        <sor>1.3</sor>
        <type>quick</type>
        <precon_iters>0</precon_iters>
        <use_dynamic_moi_rescaling>0</use_dynamic_moi_rescaling>
        <friction_model>pyramid_model</friction_model>
      </solver>
      <constraints>
        <cfm>0.0</cfm>
        <erp>0.2</erp>
        <contact_surface_layer>0.001</contact_surface_layer>
        <contact_max_correcting_vel>100.0</contact_max_correcting_vel>
      </constraints>
    </ode>
  </physics>
  <gravity>0 0 -9.8</gravity>
  <include>
    <uri>model://ground_plane</uri>
  </include>
  <include>
    <uri>model://sun</uri>
  </include>
</world>



In [15]:
# Let's change some parameters
physics.max_step_size = 0.005
physics.friction_model = 'box_model'
physics.sor = 1.5
physics.max_contacts = 10
physics.name = 'custom_ode'
print(physics.to_sdf())

<physics name="custom_ode" default="1" type="ode">
  <max_step_size>0.005</max_step_size>
  <real_time_factor>1.0</real_time_factor>
  <real_time_update_rate>1000.0</real_time_update_rate>
  <max_contacts>10</max_contacts>
  <ode>
    <solver>
      <min_step_size>0.0001</min_step_size>
      <iters>50</iters>
      <sor>1.5</sor>
      <type>quick</type>
      <precon_iters>0</precon_iters>
      <use_dynamic_moi_rescaling>0</use_dynamic_moi_rescaling>
      <friction_model>box_model</friction_model>
    </solver>
    <constraints>
      <cfm>0.0</cfm>
      <erp>0.2</erp>
      <contact_surface_layer>0.001</contact_surface_layer>
      <contact_max_correcting_vel>100.0</contact_max_correcting_vel>
    </constraints>
  </ode>
</physics>



In [16]:
# Exporting this world configuration to a file allows running the 
# configured physics engine in Gazebo
sdf = physics.to_sdf('sdf')
print(sdf)

<sdf version="1.6">
  <world name="default">
    <physics name="custom_ode" default="1" type="ode">
      <max_step_size>0.005</max_step_size>
      <real_time_factor>1.0</real_time_factor>
      <real_time_update_rate>1000.0</real_time_update_rate>
      <max_contacts>10</max_contacts>
      <ode>
        <solver>
          <min_step_size>0.0001</min_step_size>
          <iters>50</iters>
          <sor>1.5</sor>
          <type>quick</type>
          <precon_iters>0</precon_iters>
          <use_dynamic_moi_rescaling>0</use_dynamic_moi_rescaling>
          <friction_model>box_model</friction_model>
        </solver>
        <constraints>
          <cfm>0.0</cfm>
          <erp>0.2</erp>
          <contact_surface_layer>0.001</contact_surface_layer>
          <contact_max_correcting_vel>100.0</contact_max_correcting_vel>
        </constraints>
      </ode>
    </physics>
    <gravity>0 0 -9.8</gravity>
    <include>
      <uri>model://ground_plane</uri>
    </include>
    <include>
  

In [17]:
# Export to a .world file
world_filename = '/tmp/physics_ode.world'
sdf.export_xml(filename=world_filename)

In [18]:
# Remove old gazebo task
server.create_simulation('ode')
simulation = server.get_simulation('ode')
# Create new task
simulation.create_gazebo_task(
    name='gazebo',
    world=world_filename,
    gui=True,
    physics=physics.engine,
    paused=False,
    required=True,
    process_timeout=10)
# A task named 'gazebo' the added to the tasks list
print(simulation.get_task_list())
# But it is still not running
print('Is Gazebo running: {}'.format(simulation.is_task_running('gazebo')))
# Run Gazebo
simulation.run_all_tasks()

['gazebo']
Is Gazebo running: False


In [19]:
# Check if the parameters were initialized correctly
gazebo_proxy = simulation.get_gazebo_proxy()

# It is important to note that the default get_physics_properties service from Gazebo
# returns only the global and the ODE engine parameters
print(gazebo_proxy.get_physics_properties())

time_step: 0.005
pause: False
max_update_rate: 1000.0
gravity: 
  x: 0.0
  y: 0.0
  z: -9.8
ode_config: 
  auto_disable_bodies: False
  sor_pgs_precon_iters: 0
  sor_pgs_iters: 50
  sor_pgs_w: 1.5
  sor_pgs_rms_error_tol: 0.0
  contact_surface_layer: 0.001
  contact_max_correcting_vel: 100.0
  cfm: 0.0
  erp: 0.2
  max_contacts: 10
success: True
status_message: "GetPhysicsProperties: got properties"


In [20]:
simulation.wait()
simulation.kill_all_tasks()

## Bullet

In [21]:
from pcg_gazebo.simulation.physics import Bullet
physics = Bullet()

# Iterate through all parameters
for name in physics.get_parameter_names():
    print('{}: {}'.format(name, physics.get_parameter(name)))

max_step_size: 0.001
real_time_factor: 1
real_time_update_rate: 1000
max_contacts: 20
engine: bullet
name: default_physics
default: False
min_step_size: 0.0001
iters: 50
sor: 1.3
type: sequential_impulse
cfm: 0
erp: 0.2
contact_surface_layer: 0.001
split_impulse: True
split_impulse_penetration_threshold: -0.01


In [22]:
# The description of the parameters is also available
for name in physics.get_parameter_names():
    physics.print_description(name)

max_step_size
  Description: Maximum time step size at which every system in simulation can interact with the states of the world
  Current value: 0.001
real_time_factor
  Description: Target simulation speedupfactor, defined by ratio of simulation time to real-time
  Current value: 1
real_time_update_rate
  Description: Rate at which to update the physics engine (UpdatePhysics calls per real-time second)
  Current value: 1000
max_contacts
  Description: Maximum number of contactsallowed between two entities. This value can be over ridden by a max_contacts element in a collision element
  Current value: 20
engine
  Description: The type of the dynamics engine. Current options are ode, bullet, simbody and dart. Defaults to ode if left unspecified.
  Current value: bullet
name
  Description: The name of this set of physics parameters
  Current value: default_physics
default
  Description: If true, this physics element is set as the default physics profile for the world. If multiple defau

In [23]:
# It is also possible to generate the SDF files.
# The SDF object can also be retrieved and altered if necessary
print(physics.to_sdf('physics'))

<physics name="default_physics" default="1" type="bullet">
  <max_step_size>0.001</max_step_size>
  <real_time_factor>1.0</real_time_factor>
  <real_time_update_rate>1000.0</real_time_update_rate>
  <max_contacts>20</max_contacts>
  <bullet>
    <solver>
      <min_step_size>0.0001</min_step_size>
      <iters>50</iters>
      <sor>1.3</sor>
      <type>sequential_impulse</type>
    </solver>
    <constraints>
      <cfm>0.0</cfm>
      <erp>0.2</erp>
      <contact_surface_layer>0.001</contact_surface_layer>
      <split_impulse>1</split_impulse>
      <split_impulse_penetration_threshold>-0.01</split_impulse_penetration_threshold>
    </constraints>
  </bullet>
</physics>



In [24]:
print(physics.to_sdf('world'))

<world name="default">
  <physics name="default_physics" default="1" type="bullet">
    <max_step_size>0.001</max_step_size>
    <real_time_factor>1.0</real_time_factor>
    <real_time_update_rate>1000.0</real_time_update_rate>
    <max_contacts>20</max_contacts>
    <bullet>
      <solver>
        <min_step_size>0.0001</min_step_size>
        <iters>50</iters>
        <sor>1.3</sor>
        <type>sequential_impulse</type>
      </solver>
      <constraints>
        <cfm>0.0</cfm>
        <erp>0.2</erp>
        <contact_surface_layer>0.001</contact_surface_layer>
        <split_impulse>1</split_impulse>
        <split_impulse_penetration_threshold>-0.01</split_impulse_penetration_threshold>
      </constraints>
    </bullet>
  </physics>
  <gravity>0 0 -9.8</gravity>
  <include>
    <uri>model://ground_plane</uri>
  </include>
  <include>
    <uri>model://sun</uri>
  </include>
</world>



In [25]:
# Let's change some parameters
physics.max_step_size = 0.005
physics.cfm = 0.01
physics.sor = 1.5
physics.max_contacts = 5
physics.name = 'custom_bullet'
physics.real_time_update_rate = 500
print(physics.to_sdf())

<physics name="custom_bullet" default="1" type="bullet">
  <max_step_size>0.005</max_step_size>
  <real_time_factor>1.0</real_time_factor>
  <real_time_update_rate>500.0</real_time_update_rate>
  <max_contacts>5</max_contacts>
  <bullet>
    <solver>
      <min_step_size>0.0001</min_step_size>
      <iters>50</iters>
      <sor>1.5</sor>
      <type>sequential_impulse</type>
    </solver>
    <constraints>
      <cfm>0.01</cfm>
      <erp>0.2</erp>
      <contact_surface_layer>0.001</contact_surface_layer>
      <split_impulse>1</split_impulse>
      <split_impulse_penetration_threshold>-0.01</split_impulse_penetration_threshold>
    </constraints>
  </bullet>
</physics>



In [26]:
# Exporting this world configuration to a file allows running the 
# configured physics engine in Gazebo
sdf = physics.to_sdf('sdf')
print(sdf)

<sdf version="1.6">
  <world name="default">
    <physics name="custom_bullet" default="1" type="bullet">
      <max_step_size>0.005</max_step_size>
      <real_time_factor>1.0</real_time_factor>
      <real_time_update_rate>500.0</real_time_update_rate>
      <max_contacts>5</max_contacts>
      <bullet>
        <solver>
          <min_step_size>0.0001</min_step_size>
          <iters>50</iters>
          <sor>1.5</sor>
          <type>sequential_impulse</type>
        </solver>
        <constraints>
          <cfm>0.01</cfm>
          <erp>0.2</erp>
          <contact_surface_layer>0.001</contact_surface_layer>
          <split_impulse>1</split_impulse>
          <split_impulse_penetration_threshold>-0.01</split_impulse_penetration_threshold>
        </constraints>
      </bullet>
    </physics>
    <gravity>0 0 -9.8</gravity>
    <include>
      <uri>model://ground_plane</uri>
    </include>
    <include>
      <uri>model://sun</uri>
    </include>
  </world>
</sdf>



In [27]:
# Export to a .world file
world_filename = '/tmp/physics_bullet.world'
sdf.export_xml(filename=world_filename)

In [28]:
# Remove old gazebo task
server.create_simulation('bullet')
simulation = server.get_simulation('bullet')
# Create new task
simulation.create_gazebo_task(
    name='gazebo',
    world=world_filename,
    gui=True,
    physics=physics.engine,
    paused=False,
    required=True,
    process_timeout=10)
# A task named 'gazebo' the added to the tasks list
print(simulation.get_task_list())
# But it is still not running
print('Is Gazebo running: {}'.format(simulation.is_task_running('gazebo')))
# Run Gazebo
simulation.run_all_tasks()

# The get_physics_properties service does not support bullet parameters yet

['gazebo']
Is Gazebo running: False


In [29]:
simulation.wait()
simulation.kill_all_tasks()

## Simbody

In [30]:
from pcg_gazebo.simulation.physics import Simbody
physics = Simbody()

# Iterate through all parameters
for name in physics.get_parameter_names():
    print('{}: {}'.format(name, physics.get_parameter(name)))

max_step_size: 0.001
real_time_factor: 1
real_time_update_rate: 1000
max_contacts: 20
engine: simbody
name: default_physics
default: False
min_step_size: 0.0001
accuracy: 0.001
max_transient_velocity: 0.01
stiffness: 100000000.0
dissipation: 100
plastic_coef_restitution: 0.5
plastic_impact_velocity: 0.5
static_friction: 0.9
dynamic_friction: 0.9
viscous_friction: 0
override_impact_capture_velocity: 0.001
override_stiction_transition_velocity: 0.001


In [31]:
# The description of the parameters is also available
for name in physics.get_parameter_names():
    physics.print_description(name)

max_step_size
  Description: Maximum time step size at which every system in simulation can interact with the states of the world
  Current value: 0.001
real_time_factor
  Description: Target simulation speedupfactor, defined by ratio of simulation time to real-time
  Current value: 1
real_time_update_rate
  Description: Rate at which to update the physics engine (UpdatePhysics calls per real-time second)
  Current value: 1000
max_contacts
  Description: Maximum number of contactsallowed between two entities. This value can be over ridden by a max_contacts element in a collision element
  Current value: 20
engine
  Description: The type of the dynamics engine. Current options are ode, bullet, simbody and dart. Defaults to ode if left unspecified.
  Current value: simbody
name
  Description: The name of this set of physics parameters
  Current value: default_physics
default
  Description: If true, this physics element is set as the default physics profile for the world. If multiple defa

In [32]:
# It is also possible to generate the SDF files.
# The SDF object can also be retrieved and altered if necessary
print(physics.to_sdf('physics'))

<physics name="default_physics" default="1" type="simbody">
  <max_step_size>0.001</max_step_size>
  <real_time_factor>1.0</real_time_factor>
  <real_time_update_rate>1000.0</real_time_update_rate>
  <max_contacts>20</max_contacts>
  <simbody>
    <min_step_size>0.0001</min_step_size>
    <accuracy>0.001</accuracy>
    <max_transient_velocity>0.01</max_transient_velocity>
    <contact>
      <stiffness>100000000.0</stiffness>
      <dissipation>100.0</dissipation>
      <plastic_coef_restitution>0.5</plastic_coef_restitution>
      <plastic_impact_velocity>0.5</plastic_impact_velocity>
      <static_friction>0.9</static_friction>
      <dynamic_friction>0.9</dynamic_friction>
      <viscous_friction>0.0</viscous_friction>
      <override_impact_capture_velocity>0.001</override_impact_capture_velocity>
      <override_stiction_transition_velocity>0.001</override_stiction_transition_velocity>
    </contact>
  </simbody>
</physics>



In [33]:
print(physics.to_sdf('world'))

<world name="default">
  <physics name="default_physics" default="1" type="simbody">
    <max_step_size>0.001</max_step_size>
    <real_time_factor>1.0</real_time_factor>
    <real_time_update_rate>1000.0</real_time_update_rate>
    <max_contacts>20</max_contacts>
    <simbody>
      <min_step_size>0.0001</min_step_size>
      <accuracy>0.001</accuracy>
      <max_transient_velocity>0.01</max_transient_velocity>
      <contact>
        <stiffness>100000000.0</stiffness>
        <dissipation>100.0</dissipation>
        <plastic_coef_restitution>0.5</plastic_coef_restitution>
        <plastic_impact_velocity>0.5</plastic_impact_velocity>
        <static_friction>0.9</static_friction>
        <dynamic_friction>0.9</dynamic_friction>
        <viscous_friction>0.0</viscous_friction>
        <override_impact_capture_velocity>0.001</override_impact_capture_velocity>
        <override_stiction_transition_velocity>0.001</override_stiction_transition_velocity>
      </contact>
    </simbody>
  <

In [34]:
# Let's change some parameters
physics.max_step_size = 0.005
physics.max_contacts = 8
physics.name = 'custom_simbody'
physics.static_friction = 1.0
physics.real_time_update_rate = 500
print(physics.to_sdf())

<physics name="custom_simbody" default="1" type="simbody">
  <max_step_size>0.005</max_step_size>
  <real_time_factor>1.0</real_time_factor>
  <real_time_update_rate>500.0</real_time_update_rate>
  <max_contacts>8</max_contacts>
  <simbody>
    <min_step_size>0.0001</min_step_size>
    <accuracy>0.001</accuracy>
    <max_transient_velocity>0.01</max_transient_velocity>
    <contact>
      <stiffness>100000000.0</stiffness>
      <dissipation>100.0</dissipation>
      <plastic_coef_restitution>0.5</plastic_coef_restitution>
      <plastic_impact_velocity>0.5</plastic_impact_velocity>
      <static_friction>1.0</static_friction>
      <dynamic_friction>0.9</dynamic_friction>
      <viscous_friction>0.0</viscous_friction>
      <override_impact_capture_velocity>0.001</override_impact_capture_velocity>
      <override_stiction_transition_velocity>0.001</override_stiction_transition_velocity>
    </contact>
  </simbody>
</physics>



In [35]:
# Exporting this world configuration to a file allows running the 
# configured physics engine in Gazebo
sdf = physics.to_sdf('sdf')
print(sdf)

<sdf version="1.6">
  <world name="default">
    <physics name="custom_simbody" default="1" type="simbody">
      <max_step_size>0.005</max_step_size>
      <real_time_factor>1.0</real_time_factor>
      <real_time_update_rate>500.0</real_time_update_rate>
      <max_contacts>8</max_contacts>
      <simbody>
        <min_step_size>0.0001</min_step_size>
        <accuracy>0.001</accuracy>
        <max_transient_velocity>0.01</max_transient_velocity>
        <contact>
          <stiffness>100000000.0</stiffness>
          <dissipation>100.0</dissipation>
          <plastic_coef_restitution>0.5</plastic_coef_restitution>
          <plastic_impact_velocity>0.5</plastic_impact_velocity>
          <static_friction>1.0</static_friction>
          <dynamic_friction>0.9</dynamic_friction>
          <viscous_friction>0.0</viscous_friction>
          <override_impact_capture_velocity>0.001</override_impact_capture_velocity>
          <override_stiction_transition_velocity>0.001</override_stiction

In [36]:
# Export to a .world file
world_filename = '/tmp/physics_simbody.world'
sdf.export_xml(filename=world_filename)

In [37]:
server.create_simulation('simbody')
simulation = server.get_simulation('simbody')
# Create new task
simulation.create_gazebo_task(
    name='gazebo',
    world=world_filename,
    gui=True,
    physics=physics.engine,
    paused=False,
    required=True,
    process_timeout=10)
# A task named 'gazebo' the added to the tasks list
print(simulation.get_task_list())
# But it is still not running
print('Is Gazebo running: {}'.format(simulation.is_task_running('gazebo')))
# Run Gazebo
simulation.run_all_tasks()

# The get_physics_properties service does not support simbody parameters yet

['gazebo']
Is Gazebo running: False


In [38]:
simulation.wait()
simulation.kill_all_tasks()